# Compute the anomalies and merge with other data

Temperature data at monthly level from NOAA

In [1]:
import os
import psutil
import logging

import pandas as pd
import yaml
from sklearn.ensemble import RandomForestClassifier

from views.apps.ensemble import calibrate
from views.apps.ensemble import ensemble as enslib
from views.apps.osa import osa
from views.apps.plot.maps.fancy import fancy
from views.apps.model import model as modlib

from views.utils import datautils, dbutils, pyutils, config

/Users/paola/miniconda3/envs/views/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
logging.basicConfig(format=pyutils.LOGFORMAT, level=logging.INFO)
Logger = logging.getLogger(__name__)

In [3]:
import logging
import pandas as pd 
import glob
from views.utils import dbutils, datautils, pyutils

In [4]:
dbutils.recreate_schema("anomalies")
path = "/Users/paola/Documents/Data/climate" 

In [5]:
path = r'/Users/paola/Documents/Data/climate' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)

In [6]:
df = df[df['temp_mean'].notna()]

In [20]:
df['year'] = pd.to_numeric(df['year'])

In [8]:
#drop unneccessary cols
df = df.drop(columns=['Unnamed: 0'])

In [10]:
df.columns = ['lon', 'lat', 'month_year', 'temp_mean_month', 'year', 'month']

In [11]:
#store it
dbutils.df_to_db(df=df,fqtable='growseas.temperature')

[2020-05-25 16:54:13,760] - views.utils.dbutils:287 - INFO - Pushed to growseas.temperature


In [14]:
#Create ID (I do not want an Index for now)
#df['id'] = df["lon"].astype(str) + df["lat"].astype(str) + df["year"].astype(str)
#df['id2'] = df["lon"].astype(str) + df["lat"].astype(str) 
#df.sort_values(by=['id'], inplace=True)

[2020-05-25 17:06:53,408] - numexpr.utils:141 - INFO - NumExpr defaulting to 8 threads.


In [12]:
#compute annual temperature mean and standard deviation by lon, lat
df1 = df.groupby(['lon', 'lat', 'year'], as_index=False)['temp_mean_month'].mean()

In [15]:
#DF1 is df aggregated to yearly level
df1.columns = ['lon', 'lat', 'year', 'temp_mean_yearly']

In [16]:
#Add climatologic mean wrt 1930=1960

df2 = pd.read_csv("/Users/paola/Documents/Data/climate/anomalies/tempmean.csv", index_col=None)

In [18]:
#drop unneccessary cols
df2 = df2.drop(columns=['Unnamed: 0', 'mean'])

In [20]:
#Merge df1 with annual temperature and df2 with climatologic temperature by lon, lat. Left merge cause I keep all years
df3 = pd.merge (df1, df2, how = 'left', on = ['lon', 'lat'])

In [24]:
#compute standardized annual temperature anomaly vs 1930-1960
###NOTE: the reference period 1930-1960 is different than the whole period of our timeseries 
## this explains why the standardization removes some bias and noise in data 
## but cannot give a temp_std with mean 0 and std 1 
##to obtain this I would need to compute the mean across the whole timeseries

df3['temp_anom'] = (df3['temp_mean_yearly'] - df3['clima_mean'])/ df3['tempstd']

[2020-05-26 14:42:54,079] - numexpr.utils:141 - INFO - NumExpr defaulting to 8 threads.


In [29]:
#Merge  original monthly temperature data with anomalies data
df_temp = pd.merge(df, df3, how = 'left', on = ['lon', 'lat', 'year'])

In [49]:
dbutils.df_to_db(df=df_temp,fqtable='anomalies.raw')

[2020-05-26 22:47:58,977] - views.utils.dbutils:287 - INFO - Pushed to anomalies.raw


In [50]:
#Attach priogrid id

query = """
    CREATE TABLE anomalies.raw_with_pg_id AS
    SELECT priogrid(lat, lon) AS pg_id,
        anom.lon,
        anom.lat,
        anom.month_year,
        anom.year,
        anom.month,
        anom.temp_mean_month,
        anom.temp_mean_yearly,
        anom.clima_mean,
        anom.tempstd,
        anom.temp_anom
        FROM anomalies.raw AS anom;
    """
dbutils.execute_query(query)


In [59]:
#now attach pgm-MONTH id reference
query = """
    DROP TABLE if exists anomalies.attached;
    CREATE TABLE anomalies.attached AS
        SELECT pgm.id AS priogrid_month_id,
               pgm.year_id AS year,
               anom.lon,
               anom.pg_id,
               anom.lat,
               anom.month_year,
               anom.month,
               anom.temp_mean_month,
               anom.temp_mean_yearly,
               anom.clima_mean,
               anom.tempstd,
               anom.temp_anom
    FROM staging.month AS pgm
    LEFT JOIN anomalies.raw_with_pg_id AS anom ON pgm.month=anom.month AND pgm.year_id = anom.year;
    """
dbutils.execute_query(query)

In [62]:
#attach month and other references (year id, country month...)
query = """
    CREATE TABLE anomalies.month AS
        SELECT pgm.id AS priogrid_month_id,
               pgm.priogrid_gid AS pg_id,
               pgm.month_id AS month,
               pgm.priogrid_year_id,
               pgm.country_month_id,  
               anom.lon,
               anom.lat,
               anom.month_year,
               anom.year,
               anom.temp_mean_month,
               anom.temp_mean_yearly,
               anom.clima_mean,
               anom.tempstd,
               anom.temp_anom
    FROM staging.priogrid_month AS pgm
    LEFT JOIN anomalies.attached AS anom ON pgm.priogrid_gid=anom.pg_id AND pgm.month_id = anom.month;
    """
dbutils.execute_query(query)

In [69]:
#finally merge with the other data in a view
query = """
    DROP TABLE IF EXISTS growseas.climate_anomalies;
    CREATE VIEW growseas.climate_anomalies AS
        SELECT gs.*,
               anom.lon,
               anom.lat,
               anom.month_year,
               anom.year,
               anom.month,
               anom.temp_mean_month,
               anom.temp_mean_yearly,
               anom.clima_mean,
               anom.tempstd,
               anom.temp_anom     
        FROM growseas.climate AS gs
        LEFT JOIN anomalies.attached AS anom ON gs.priogrid_month_id = anom.priogrid_month_id;
        """
dbutils.execute_query(query)